In [1]:
!pip install -q transformers
!pip install -q simpletransformers

     |████████████████████████████████| 4.0 MB 5.6 MB/s 
     |████████████████████████████████| 895 kB 27.2 MB/s 
     |████████████████████████████████| 596 kB 37.1 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 6.6 MB 38.5 MB/s 
     |████████████████████████████████| 249 kB 5.2 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 1.2 MB 50.2 MB/s 
     |████████████████████████████████| 325 kB 52.5 MB/s 
     |████████████████████████████████| 1.8 MB 36.3 MB/s 
     |████████████████████████████████| 10.1 MB 40.1 MB/s 
     |████████████████████████████████| 181 kB 51.6 MB/s 
     |████████████████████████████████| 144 kB 54.0 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 1.1 MB 40.7 MB/s 
     |████████████████████████████████| 212 kB 53.7 MB/s 
     |████████████████████████████████| 136 kB 56.5 MB/s 
     |███████████████

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# SARCASM DETECTION

In [ ]:
import torch
model_save_name = 'sarcasm.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
model = torch.load(path,map_location=torch.device('cpu'))

In [ ]:
samples = ["I totally understand man! 😏",'I am going to kill you man 🤣','I will kill you nigga',"Ohhhh she's like...soooo pretty🤣","they atleast don 't make you feel socially unaccepted # hahahaha # lol # truestory " ]
predictions, _ = model.predict(samples) 
label_dict = {0: 'not sarcastic', 1: 'sarcastic'}
ff = ['🙂','😂','🤣','😌','😒','😏','💩','🤭','👀','🍌','🍆','🍒','🍑','🚩','🌝','🌚']
for idx, sample in enumerate(samples):
  f=0
  for j in ff:
    if j in sample:
      print('{}: {} ===> {}'.format(idx, sample, 'sarcastic'))
      f=1
  if f==0:
    print('{}: {} ===> {}'.format(idx, sample, label_dict[predictions[idx]]))


# SEXISM DETECTION

In [3]:
import pandas as pd
import numpy as np
import math as m
import nltk 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
import os
import math
import torch
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from transformers import XLNetTokenizerFast

%matplotlib inline
nltk.download('stopwords')
nltk.download ('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [14]:
def load_model(save_path):
  """
  Load the model from the path directory provided
  """
  checkpoint = torch.load(save_path)
  model_state_dict = checkpoint['state_dict']
  model = XLNetForMultiLabelSequenceClassification(num_labels=model_state_dict["classifier.weight"].size()[0])
  model.load_state_dict(model_state_dict)

  epochs = checkpoint["epochs"]
  lowest_eval_loss = checkpoint["lowest_eval_loss"]
  train_loss_hist = checkpoint["train_loss_hist"]
  valid_loss_hist = checkpoint["valid_loss_hist"]
  
  return model, epochs, lowest_eval_loss, train_loss_hist, valid_loss_hist  

In [15]:
torch.cuda.empty_cache()

In [16]:
class XLNetForMultiLabelSequenceClassification(torch.nn.Module):
  
  def __init__(self, num_labels=2):
    super(XLNetForMultiLabelSequenceClassification, self).__init__()
    self.num_labels = num_labels
    self.xlnet = XLNetModel.from_pretrained('xlnet-base-cased')
    self.classifier = torch.nn.Linear(768, num_labels)

    torch.nn.init.xavier_normal_(self.classifier.weight)

  def forward(self, input_ids, token_type_ids=None,\
              attention_mask=None, labels=None):
    # last hidden layer
    last_hidden_state = self.xlnet(input_ids=input_ids,\
                                   attention_mask=attention_mask,\
                                   token_type_ids=token_type_ids)
    # pool the outputs into a mean vector
    mean_last_hidden_state = self.pool_hidden_state(last_hidden_state)
    logits = self.classifier(mean_last_hidden_state)
        
    if labels is not None:
      loss_fct = BCEWithLogitsLoss()
      loss = loss_fct(logits.view(-1, self.num_labels),\
                      labels.view(-1, self.num_labels))
      return loss
    else:
      return logits
    
  def freeze_xlnet_decoder(self):
    """
    Freeze XLNet weight parameters. They will not be updated during training.
    """
    for param in self.xlnet.parameters():
      param.requires_grad = False
    
  def unfreeze_xlnet_decoder(self):
    """
    Unfreeze XLNet weight parameters. They will be updated during training.
    """
    for param in self.xlnet.parameters():
      param.requires_grad = True
    
  def pool_hidden_state(self, last_hidden_state):
    """
    Pool the output vectors into a single mean vector 
    """
    last_hidden_state = last_hidden_state[0]
    mean_last_hidden_state = torch.mean(last_hidden_state, 1)
    return mean_last_hidden_state
    

In [19]:
device = torch.device("cuda")
model_save_name = 'detection.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
model = XLNetForMultiLabelSequenceClassification()
model.load_state_dict(torch.load(path))
model.to(device)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


XLNetForMultiLabelSequenceClassification(
  (xlnet): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedFor

In [21]:
def generate_predictions(model, df, device="cpu", batch_size=16):
  num_iter = math.ceil(df.shape[0]/batch_size)
  
  pred_probs = []

  #model.to(device)
  #model.eval()
  
  for i in range(num_iter):
    df_subset = df.iloc[i*batch_size:(i+1)*batch_size,:]
    X = df_subset["features"].values.tolist()
    masks = df_subset["masks"].values.tolist()
    X = torch.tensor(X)
    masks = torch.tensor(masks, dtype=torch.long)
    X = X.to(device)
    masks = masks.to(device)
    with torch.no_grad():
      logits = model(input_ids=X, attention_mask=masks)
      logits = logits.sigmoid().detach().cpu().numpy()
#       pred_probs = np.vstack([pred_probs, logits])
      pred_probs.extend(logits.tolist())
        
  return pred_probs

In [31]:
from transformers import XLNetTokenizerFast
tokenizer = XLNetTokenizerFast.from_pretrained('xlnet-base-cased', do_lower_case=True)
def tokenize_inputs(text_list, tokenizer, num_embeddings=512):
    tokenized_texts = list(map(lambda t: tokenizer.tokenize(t)[:num_embeddings-2], text_list))
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = [tokenizer.build_inputs_with_special_tokens(x) for x in input_ids]
    input_ids = pad_sequences(input_ids, maxlen=num_embeddings, dtype="long", truncating="post", padding="post")
    return input_ids

def create_attn_masks(input_ids):
    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return attention_masks

input = ["you are a fucking slut! Go suck some dick!","faggots in this webspace are over-smart"]
valid = pd.DataFrame()
valid['HATESPEECH'] = input
validid = tokenize_inputs(input, tokenizer, num_embeddings=250)
attention_masks = create_attn_masks(validid)
valid["features"] = validid.tolist()
valid["masks"] = attention_masks
valid.head()

,HATESPEECH,features,masks
0,you are a fucking slut! Go suck some dick!,"[44, 41, 24, 19021, 17, 23, 2311, 46, 136, 147...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
1,faggots in this webspace are over-smart,"[17, 2819, 5723, 13208, 25, 52, 1015, 8491, 41...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."


In [32]:
num_labels = 2
pred_probs = generate_predictions(model, valid, device="cuda", batch_size=16)
pred_probs

[[0.00787911843508482, 0.9843763113021851],
 [0.9999620914459229, 2.3915090423543006e-05]]

In [33]:
u = list(pred_probs)
v = []
vv = []
for i in u:
  if i[0] < 0.40 and i[1]<0.40:
    v.append(0)
    vv.append("Clear")
  elif i[0]>i[1]:
    v.append(round(i[0],4))
    vv.append("LGBT+")
  else:
    v.append(round(i[1],4))
    vv.append("WOMEN+")

In [34]:
uu = valid['HATESPEECH']
ans_frame = pd.DataFrame()
ans_frame['text'] = uu
ans_frame['tag'] = vv
ans_frame['probability'] = v

In [35]:
ans_frame

,text,tag,probability
0,you are a fucking slut! Go suck some dick!,WOMEN+,0.9844
1,faggots in this webspace are over-smart,LGBT+,1.0000


# PARAPHRASING || PEGASUS

In [37]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [46]:
num_beams = 10
num_return_sequences = 10
context = input
for j in context:
  u = get_response(j,num_return_sequences,num_beams)
  for i in u:
    i = i.replace('slut','person')
    i = i.replace('whore','person')
    i = i.replace('faggot','person')
    i = i.replace('dyke','person')
    print(i)
  print("**************************************")

You are a person!
You are a person.
You're a person!
Go suck some dick!
Go suck some shit!
You are a disgusting person!
You are a person, go suck some dick!
You are a personty!
You are a person! Go and have a good time!
You are a person, go suck some shit!
**************************************
The persons are smart.
The persons in this webspace are smart.
The persons in this space are smart.
The persons in this webspace are very smart.
This webspace has persons that are over-smart.
There are persons in this space.
This webspace has persons that are over smart.
There are persons in this webspace.
The persons in this space are very smart.
This webspace has persons that are smart.
**************************************
